Jonathan Sosa  

Jun - July 2023

Stock Predictor


In [2]:
# Imports
import pandas as pd
import matplotlib as plot
import yfinance as yf

# DATA COLLECTION
Fetch data from Yahoo Fiance from different stocks

In [10]:
apple_dta = yf.download('AAPL', start='2010-01-01', end='2023-05-01')
meta_dta = yf.download('META', start='2010-01-01', end='2023-05-01')
nike_dta = yf.download('NKE', start='2010-01-01', end='2023-05-01')
smp500_dta = yf.download('^GSPC', start='2010-01-01', end='2023-05-01')
vanguard_dta = yf.download('VOO', start='2010-01-01', end='2023-05-01')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [12]:
print(apple_dta.head())
print(meta_dta.head())
print(nike_dta.head())
print(smp500_dta.head())
print(vanguard_dta.head())

dta = [apple_dta, meta_dta, nike_dta, smp500_dta, vanguard_dta]

                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2010-01-04  7.622500  7.660714  7.585000  7.643214   6.496295  493729600
2010-01-05  7.664286  7.699643  7.616071  7.656429   6.507524  601904800
2010-01-06  7.656429  7.686786  7.526786  7.534643   6.404016  552160000
2010-01-07  7.562500  7.571429  7.466071  7.520714   6.392176  477131200
2010-01-08  7.510714  7.571429  7.466429  7.570714   6.434674  447610800
                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2012-05-18  42.049999  45.000000  38.000000  38.230000  38.230000  573576400
2012-05-21  36.529999  36.660000  33.000000  34.029999  34.029999  168192700
2012-05-22  32.610001  33.590000  30.940001  31.000000  31.000000  101786600
2012-05-23  31.370001  32.500000  31.360001  32.000000  32.000000   73600000
2012-05-24  32.950001  33.2

# DATA PROCESSING
handling missing values, converting data types, and creating new features

In [19]:
for d in dta:
    # Add a feature for the previous day's close price
    d['PrevClose'] = d['Close'].shift(1)

    # Add a feature for the change in close price from the previous day
    d['PriceChange'] = d['Close'] - d['PrevClose']
    
    print(d.head())

                Open      High       Low     Close  Adj Close     Volume  \
Date                                                                       
2010-01-04  7.622500  7.660714  7.585000  7.643214   6.496295  493729600   
2010-01-05  7.664286  7.699643  7.616071  7.656429   6.507524  601904800   
2010-01-06  7.656429  7.686786  7.526786  7.534643   6.404016  552160000   
2010-01-07  7.562500  7.571429  7.466071  7.520714   6.392176  477131200   
2010-01-08  7.510714  7.571429  7.466429  7.570714   6.434674  447610800   

            PrevClose  PriceChange  
Date                                
2010-01-04        NaN          NaN  
2010-01-05   7.643214     0.013215  
2010-01-06   7.656429    -0.121786  
2010-01-07   7.534643    -0.013929  
2010-01-08   7.520714     0.050000  
                 Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2012-05-18  42.049999  45.000000  38.000000  38.

In [17]:
# Check for missing values
for d in dta:
    print(d.isnull().sum())
   
    # This propagates the last valid observation forward to next valid.
    d.fillna(method='ffill', inplace=True)

Open           0
High           0
Low            0
Close          0
Adj Close      0
Volume         0
PrevClose      1
PriceChange    1
dtype: int64
Open           0
High           0
Low            0
Close          0
Adj Close      0
Volume         0
PrevClose      1
PriceChange    1
dtype: int64
Open           0
High           0
Low            0
Close          0
Adj Close      0
Volume         0
PrevClose      1
PriceChange    1
dtype: int64
Open           0
High           0
Low            0
Close          0
Adj Close      0
Volume         0
PrevClose      1
PriceChange    1
dtype: int64
Open           0
High           0
Low            0
Close          0
Adj Close      0
Volume         0
PrevClose      1
PriceChange    1
dtype: int64


In [20]:
# Define the size of the test set
test_size = 0.2  # 20% of the data

def split_data(data, test_size):
    '''
    This function takes the data and divides it into test and train datasets
    ----------------
    Inputs:
    data: pandas df
    test_size: float
    -----------------
    Outputs:
    train_data: pandas df
    test_data: pandas df
    '''
    
    # Calculate the index at which to split the data
    split_idx = int(len(data) * (1 - test_size))

    # Split the data
    train_data = data[:split_idx]
    test_data = data[split_idx:]


    return (train_data, test_data)


In [ ]:
apple_dta_train, apple_dta_test = split_data(apple_dta, test_size)
meta_dta_train, meta_dta_test = split_data(meta_dta, test_size) 
nike_dta_train, nike_dta_test = split_data(nike_dta, test_size) 
smp500_dta_train, smp500_dta_test = split_data(smp500_dta, test_size)  
vanguard_dta_train, vanguard_dta_test = split_data(vanguard_dta, test_size) 